# Fully Connecter Network With Pytorch

[Fully-Connected-Neural-Network-MNIST-Classification-PyTorch](https://github.com/milindmalshe/Fully-Connected-Neural-Network-MNIST-Classification-PyTorch/blob/master/FCN_MNIST_Classification_PyTorch.py)

## Import libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Check Cuda configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Define Hyper-parameters

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## Get dataset: MNIST

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())


100.1%

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


113.5%

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


100.4%

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


180.4%

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


## Data loader

In [5]:

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Model: Fully connected neural network 

In [9]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## Define Loss

In [10]:
criterion = nn.CrossEntropyLoss()

## Define Optimizers

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Training loop

In [12]:

total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3644
Epoch [1/5], Step [200/600], Loss: 0.2016
Epoch [1/5], Step [300/600], Loss: 0.2095
Epoch [1/5], Step [400/600], Loss: 0.2520
Epoch [1/5], Step [500/600], Loss: 0.1950
Epoch [1/5], Step [600/600], Loss: 0.1465
Epoch [2/5], Step [100/600], Loss: 0.0383
Epoch [2/5], Step [200/600], Loss: 0.1089
Epoch [2/5], Step [300/600], Loss: 0.0811
Epoch [2/5], Step [400/600], Loss: 0.0722
Epoch [2/5], Step [500/600], Loss: 0.1260
Epoch [2/5], Step [600/600], Loss: 0.1419
Epoch [3/5], Step [100/600], Loss: 0.0793
Epoch [3/5], Step [200/600], Loss: 0.0613
Epoch [3/5], Step [300/600], Loss: 0.1226
Epoch [3/5], Step [400/600], Loss: 0.0530
Epoch [3/5], Step [500/600], Loss: 0.0778
Epoch [3/5], Step [600/600], Loss: 0.0338
Epoch [4/5], Step [100/600], Loss: 0.0392
Epoch [4/5], Step [200/600], Loss: 0.0632
Epoch [4/5], Step [300/600], Loss: 0.0258
Epoch [4/5], Step [400/600], Loss: 0.0191
Epoch [4/5], Step [500/600], Loss: 0.0350
Epoch [4/5], Step [600/600], Loss:

## Get Accuracy: run model over Test data 

In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.94 %


## Save checkpoint

In [14]:
torch.save(model.state_dict(), 'model.ckpt')